# 📊 Crypto AI System - Comprehensive Visual Report

This notebook visualizes the key metrics and performance indicators of the Crypto AI Decision System.

**Sections:**
1.  **Performance Analytics**: Equity Curve, Drawdown, Trade Analysis.
2.  **Data Drift**: Population Stability Index (PSI) for features.
3.  **RL Simulation**: Training progress and agent equity.
4.  **Market Regimes**: Price action colored by detected regimes.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

# Config
sns.set_style("darkgrid")
plt.rcParams["figure.figsize"] = (15, 8)

# Paths
PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
LOGS_DIR = DATA_DIR / "execution"
ANALYTICS_DIR = DATA_DIR / "analytics"
RESEARCH_DIR = DATA_DIR / "research"
FEATURES_DIR = DATA_DIR / "features"

ModuleNotFoundError: No module named 'seaborn'

## 1. Performance Analytics

In [ ]:
def load_trading_logs():
    log_file = LOGS_DIR / "paper_trades.jsonl"
    if not log_file.exists():
        print(f"Log file not found: {log_file}")
        return pd.DataFrame()
    
    data = []
    with open(log_file, "r") as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except:
                continue
    
    df = pd.DataFrame(data)
    if "timestamp" in df.columns:
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df = df.sort_values("timestamp")
    return df

df_trades = load_trading_logs()

if not df_trades.empty:
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    
    # Equity Curve
    ax1.plot(df_trades["timestamp"], df_trades["equity"], label="Equity", color="#00ff00")
    ax1.set_title("Equity Curve", fontsize=16)
    ax1.set_ylabel("Equity ($)")
    ax1.legend()
    
    # Drawdown
    rolling_max = df_trades["equity"].cummax()
    drawdown = (df_trades["equity"] - rolling_max) / rolling_max * 100
    
    ax2.fill_between(df_trades["timestamp"], drawdown, 0, color="#ff0000", alpha=0.3, label="Drawdown %")
    ax2.set_title("Drawdown", fontsize=14)
    ax2.set_ylabel("%")
    ax2.set_xlabel("Date")
    
    plt.tight_layout()
    plt.show()
else:
    print("No trading data available.")

## 2. Live Drift Detection

In [ ]:
drift_file = ANALYTICS_DIR / "drift_report.csv"

if drift_file.exists():
    df_drift = pd.read_csv(drift_file)
    
    # Filter for top drift
    top_drift = df_drift.sort_values("psi", ascending=False).head(10)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(data=top_drift, x="psi", y="feature", hue="status", dodge=False)
    plt.title("Top 10 Features by Drift (PSI)", fontsize=16)
    plt.xlabel("Population Stability Index (PSI)")
    plt.axvline(0.1, color="orange", linestyle="--", label="Warning (0.1)")
    plt.axvline(0.2, color="red", linestyle="--", label="Critical (0.2)")
    plt.legend()
    plt.show()
else:
    print("No drift report found.")

## 3. Reinforcement Learning Simulation

In [ ]:
rl_file = RESEARCH_DIR / "rl_results.csv"

if rl_file.exists():
    df_rl = pd.read_csv(rl_file)
    
    plt.figure(figsize=(12, 6))
    plt.plot(df_rl.index, df_rl["equity"], color="purple", label="RL Agent")
    plt.title("RL Agent Training Performance", fontsize=16)
    plt.ylabel("Equity ($)")
    plt.xlabel("Steps")
    plt.legend()
    plt.show()
else:
    print("No RL results found.")

## 4. Market Regimes

In [ ]:
features_file = FEATURES_DIR / "features_1H_advanced.parquet"

if features_file.exists():
    df_feat = pd.read_parquet(features_file)
    # Use last 500 candles for clarity
    df_subset = df_feat.iloc[-500:].copy()
    
    # Map regimes if available (assuming 'regime' column, else mock)
    # If regime not in columns, we skip coloring
    
    plt.figure(figsize=(15, 8))
    plt.plot(df_subset["timestamp"], df_subset["btc_close"], label="Price", color="black", alpha=0.5)
    
    if "regime" in df_subset.columns:
        # Scatter plot for regimes
        sns.scatterplot(data=df_subset, x="timestamp", y="btc_close", hue="regime", palette="deep", s=20)
        
    plt.title("BTC Price Action & Market Regimes", fontsize=16)
    plt.ylabel("Price ($)")
    plt.legend()
    plt.show()
else:
    print("No feature data found.")